In [9]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client, metrics, wait
# wait for jobs to arrive, depending on the queue, this may take some time
import dask.array as da
import dask.bag as db
import numpy as np
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
from distributed import progress
import os
from matplotlib import use
use('agg')
os.environ['PROJ_LIB'] = '/home/zsherman/anaconda3/envs/cmac_env/share/proj/'
import pyart
import netCDF4
import tarfile
import tempfile
import shutil
from netCDF4 import num2date
import json
from time import strftime, sleep
import os
%matplotlib inline
import argparse
import datetime
import glob
import importlib
import subprocess
import time

from cmac import (cmac, get_cmac_values, quicklooks,
                  get_sounding_times, get_sounding_file_name)

In [2]:
def run_cmac_and_plotting(
    radar_file_path, sounding_times, config, clutter_file, out_radar, image_directory, bad_directory, meta_append,
    overwrite, dd_lobes):
    """ For dask we need the radar plotting routines all in one subroutine. """
    cmac_config = get_cmac_values(config)

    try:
        radar = pyart.io.read(radar_file_path)
    except TypeError:
        if bad_directory is None:
            path = os.path.expanduser('~') + '/' + 'type_error_radars/'
        else:
            path = bad_directory
        print(radar_file_path + ' has encountered TypeError!')
        if not os.path.exists(path):
            os.makedirs(path)
            subprocess.call('chmod -R g+rw ' + path, shell=True)
        shutil.move(radar_file_path, path)
        return

    radar_start_date = netCDF4.num2date(radar.time['data'][0],
                                        radar.time['units'])
    year_str = "%04d" % radar_start_date.year
    month_str = "%02d" % radar_start_date.month
    day_str = "%02d" % radar_start_date.day
    hour_str = "%02d" % radar_start_date.hour
    minute_str = "%02d" % radar_start_date.minute
    second_str = "%02d" % radar_start_date.second

    save_name = cmac_config['save_name']
    if out_radar is None:
        the_path = (os.path.expanduser('~') + '/'+ year_str + month_str
                    + second_str + '/')
    else:
        the_path = (out_radar + '/' + year_str +  month_str
                    + day_str)
    file_name = (the_path + '/' + save_name + '.'
                 + year_str + month_str + day_str + '.' + hour_str
                 + minute_str + second_str + '.nc')

    # If overwrite is False, checks to see if the cmac_radar file
    # already exists. If so, CMAC 2.0 is not used on the original radar file.
    if overwrite is False and os.path.exists(file_name) is True:
        print(file_name + ' already exists.')
        return

    if not os.path.exists(the_path):
        os.makedirs(the_path)
        subprocess.call('chmod -R g+rw ' + the_path, shell=True)

    # Load clutter files.
    if clutter_file is not None:
        clutter_file_path = clutter_file
        if verbose:
            print('## Loading clutter file ' + clutter_file_path)
        clutter = pyart.io.read(clutter_file_path)
        if verbose:
            print('## Reading dictionary...')
        clutter_field_dict = clutter.fields['xsapr_clutter']
        if verbose:
            print('## Adding clutter field..')
        try:
            radar.add_field(
                'xsapr_clutter', clutter_field_dict, replace_existing=True)
            del clutter
        except ValueError:
            del clutter
            return
    # Retrieve closest sonde in time to the time of the radar file.
    closest_time = min(
        sounding_times, key=lambda d: abs(d - radar_start_date))
    sonde_name = cmac_config['sonde_name']
    sonde_file = get_sounding_file_name(
        sonde_path, sonde_name, closest_time)
    sonde = netCDF4.Dataset(sonde_file)

    # Running the cmac code to produce a cmac_radar object.
    try:
        cmac_radar = cmac(radar, sonde, config, flip_velocity=True,
                          meta_append=meta_append,
                          verbose=verbose)
    except ValueError:
        del radar
        sonde.close()
        return

    # Free up some memory.
    del radar
    sonde.close()

    # Produce the cmac_radar file from the cmac_radar object.
    pyart.io.write_cfradial(file_name, cmac_radar)
    print('## A CMAC radar object has been created at ' + file_name)

    # Providing the image_directory and checking if it already exists.
    img_directory = (image_directory + '/' + year_str + month_str
                     + day_str + '.' + hour_str + minute_str + second_str)
    if not os.path.exists(img_directory):
        os.makedirs(img_directory)
        subprocess.call('chmod -R g+rw ' + img_directory, shell=True)

    # Producing all the cmac_radar quicklooks.
    quicklooks(cmac_radar, config,
               image_directory=img_directory,
               dd_lobes=dd_lobes)

    # Delete the cmac_radar object and move on to the next radar file.
    del cmac_radar
    return

In [3]:
radar_path = '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/'
sonde_path = '/lustre/or-hydra/cades-arm/proj-shared/sgpsondewnpnC1.b1/'
config = 'xsapr_i4_ppi'

In [4]:
cmac_config = get_cmac_values(config)

x_compass = cmac_config['x_compass']
if os.path.isdir(radar_path):
    radar_files = glob.glob(radar_path + '/**/*' + x_compass
                            + '*', recursive=True)
elif os.path.isfile(radar_path):
    with open(radar_path) as f:
        radar_files = f.readlines()
    radar_files = [x.strip() for x in radar_files]
else:
    raise IOError('The specified radar path does not exist!')
sonde_name = cmac_config['sonde_name']
sounding_times = get_sounding_times(sonde_path, sonde_name)

# Get dates of radar files from the file name.
radar_times = []
for file_name in radar_files:
    where_x = file_name.find(x_compass)
    radar_times.append(
        datetime.datetime.strptime(file_name[where_x+3:where_x+15],
                                   '%y%m%d%H%M%S'))

In [5]:
radar_files

['/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/XSE171004225509.RAWW41D',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/XSE171002042424.RAWW3T4',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/XSE171005054433.RAWW428',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/sgpxsaprI4.00.20171030.191001.mnt.XSE171030175016.RAW74KN.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/sgpxsaprI4.00.20171011.031001.mnt.XSE171011021751.RAWULHF.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/sgpxsaprI4.00.20171026.051001.mnt.XSE171026040811.RAW6Z6M.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/sgpxsaprI4.00.20171009.231001.mnt.XSE171009222723.RAWULE0.maint',
 '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/201710/sgpxsaprI4.00.20171024.131001.mnt.XSE171024124008.RAW6Z1S.maint',
 '/lustre/or-hydra/cades-arm/proj-shared

In [6]:
bad_directory = None
out_radar = '/lustre/or-hydra/cades-arm/proj-shared/sgpxsaprcmacsurI4.c1'
image_directory = '/lustre/or-hydra/cades-arm/proj-shared/sgpxsaprcmacsurI4.c1.png'
meta_append = 'config'
overwrite = True
clutter_file = '/lustre/or-hydra/cades-arm/proj-shared/raw_sgpxsaprppiI4.00/clutter_files/clutter_201710.nc'
verbose = False
dd_lobes = True

In [7]:
#cluster = PBSCluster(name='dask-worker', memory='270GB', cores=36, processes=6, interface='ib0', queue='high_mem', project='arm',
#                    walltime='00:30:00')#, job-extra=['-W group_list=cades-arm'])
cluster1 = PBSCluster(processes=12, cores=12, walltime='3:00:00', queue='batch', memory='128GB')
cluster1.scale(12*12)         # Ask for ten workers
client1 = Client(cluster1)  # Connect this local process to remote workers

/home/zsherman/anaconda3/envs/cmac_env/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [10]:
cluster1

In [11]:
client1

Client Scheduler: tcp://10.23.216.82:34329 Dashboard: http://10.23.216.82:43792/status,Cluster Workers: 144 Cores: 144 Memory: 1.54 TB


In [12]:
def com():
    from matplotlib import use
    use('agg')
client1.run(com)

{'tcp://10.23.216.202:33627': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:36098': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:36743': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:38402': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:38710': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:39900': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:40277': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:42296': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:44092': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:45585': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:45816': '/home/zsherman/anaconda3/envs/cmac_env/share/proj/',
 'tcp://10.23.216.202:46784': '/home/zsherman/anaconda3/envs/cmac_env/share/

In [13]:
#the_bag = db.from_sequence(radar_files)
the_function = lambda x: run_cmac_and_plotting(
    x, sounding_times=sounding_times, config='xsapr_i4_ppi',
    clutter_file=clutter_file, out_radar=out_radar, image_directory=image_directory,
    bad_directory=bad_directory, meta_append=meta_append,
    overwrite=overwrite, dd_lobes=dd_lobes)
futures1 = client1.map(the_function, radar_files)

In [14]:
#start computation in the background
progress(futures1)

VBox()

In [15]:
cluster1.stop_all_jobs()